In [2]:
checkpoint_file = "gs://transformers-cnn/results/iclr/gauss_l6_h9_d576_batch100_e100_noniso/0099.checkpoint"

In [5]:
import torch
from tensorflow.python.lib.io import file_io

# download parameters from checkpoint
with file_io.FileIO(checkpoint_file, 'rb') as f:
    checkpoint_data = torch.load(f, map_location='cpu')

In [7]:
parameters = checkpoint_data["model_state_dict"]

In [21]:
import re

attention_centers = []
attention_spreads = []

for name, param in parameters.items():
    res = re.findall("encoder.layer.(\d+).attention.self.attention_(.+)", name)
    if res:
        layer_num, typ = int(res[0][0]), res[0][1]
        if typ == 'centers':
            attention_centers.append(param)
        elif typ == 'spreads':
            attention_spreads.append(param)
        else:
            print("unknown type '{}'".format(typ))
            
# -- n_layers, n_head, 2
attention_centers = torch.stack(attention_centers)

# -- n_layers, n_head, 2, 2
attention_spreads = torch.stack(attention_spreads)

In [29]:
import json

data = dict(
    n_layers = attention_centers.shape[0],
    n_head = attention_centers.shape[1],
    attention_centers = attention_centers.numpy().tolist(),
    attention_spreads = attention_spreads.numpy().tolist()
)

with open("export_attention.json", "w") as f:
    json.dump(data, f)